In [1]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
random_state = 42
np.random.seed(random_state)

In [2]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv') 
data = pd.concat(([ctr_15,ctr_16]))
print("Tamaño del conjunto ", data.shape)
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
print("Tamaño del conjunto ", ctr_test.shape)

Tamaño del conjunto  (2331937, 52)
Tamaño del conjunto  (1139639, 52)


In [3]:
# Lista de columnas a eliminar
columns_to_remove = [
    'device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 
    'auction_list_0', 'action_categorical_2', 'action_categorical_4', 
    'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 
    'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6',
    'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10',
    'auction_categorical_4', 'gender', 'creative_categorical_1', 
    'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3',
    'auction_categorical_2', 'action_categorical_5', 'action_list_0',
    'auction_categorical_1', 'auction_categorical_5', 'device_id_type',
    'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0',
    'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5',
    'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8',
    'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1',
    'creative_categorical_2', 'action_categorical_3', 'action_categorical_2',
    'auction_categorical_11', 'action_categorical_4',
    'creative_categorical_0', 'device_id'
]

data = data.drop(columns=columns_to_remove, errors='ignore')
ctr_test = ctr_test.drop(columns=columns_to_remove, errors='ignore')

id_col_test = ctr_test["id"]
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print(f"Columnas eliminadas: {columns_to_remove}")
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")


Columnas eliminadas: ['device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 'auction_list_0', 'action_categorical_2', 'action_categorical_4', 'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6', 'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10', 'auction_categorical_4', 'gender', 'creative_categorical_1', 'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3', 'auction_categorical_2', 'action_categorical_5', 'action_list_0', 'auction_categorical_1', 'auction_categorical_5', 'device_id_type', 'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0', 'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5', 'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8', 'creative_categorical_4', 'creative_categorical_6', 'act

In [4]:
def group_rare_categories(column, threshold):
    freq = column.value_counts(normalize=True)
    rare_categories = freq[freq < threshold].index
    return column.apply(lambda x: 'Other' if x in rare_categories else x)

threshold = 0.05

categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
categorical_columns_eval = ctr_test.select_dtypes(include=['object']).columns.tolist()

#categorical_columns = ['auction_categorical_12', 'auction_categorical_3', 'auction_categorical_7', 'auction_categorical_9']
#list_columns = ['action_list_2', 'action_list_1', 'auction_list_0']

for col in categorical_columns:
    data[col] = group_rare_categories(data[col], threshold)

for col in categorical_columns_eval:
    ctr_test[col] = group_rare_categories(ctr_test[col], threshold)
pd_ohe = pd.get_dummies(data,
                        columns=categorical_columns + ['has_video'],  # Incluye has_video también
                        sparse=False,    # Devolver una matriz no rala para facilitar la alineación
                        dummy_na=False,  # No agregar columna para NaNs.
                        dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )

pd_ohe_eval = pd.get_dummies(ctr_test,
                             columns=categorical_columns_eval + ['has_video'],  # Incluye has_video también
                             sparse=False,    # Devolver una matriz no rala
                             dummy_na=False,  # No agregar columna para NaNs.
                             dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                            )

pd_ohe_eval = pd_ohe_eval.reindex(columns=pd_ohe.columns, fill_value=0)

pd_ohe_eval['id'] = id_col_test.values

print("Shape de data_drop después de OneHotEncoding:", pd_ohe.shape)
print("Shape de eval_data_drop después de OneHotEncoding:", pd_ohe_eval.shape)

pd_ohe.info()
pd_ohe_eval.info()

Shape de data_drop después de OneHotEncoding: (2331937, 17)
Shape de eval_data_drop después de OneHotEncoding: (1139639, 18)
<class 'pandas.core.frame.DataFrame'>
Index: 2331937 entries, 0 to 1092412
Data columns (total 17 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Label                            int64  
 1   auction_age                      float64
 2   auction_bidfloor                 float64
 3   auction_time                     float64
 4   creative_height                  float64
 5   creative_width                   float64
 6   timezone_offset                  float64
 7   auction_categorical_12_9c3324c7  int64  
 8   auction_categorical_12_Other     int64  
 9   auction_categorical_12_b4e520dd  int64  
 10  auction_categorical_12_e9e0850e  int64  
 11  auction_categorical_9_11105871   int64  
 12  auction_categorical_9_1b26c761   int64  
 13  auction_categorical_9_Other      int64  
 14  auction_categorical_9_d4ee

In [5]:
y = pd_ohe[['Label']].copy() # Usamos copy para no modificar un view abajo, ya que genera un warning.
X = pd_ohe.drop('Label', axis = 1)

X_train, X_val, Y_train, Y_val = train_test_split(X, y,
                                                  train_size = 0.8,
                                                  random_state = 32,
                                                  stratify = y)

X_test = pd_ohe_eval.drop('Label', axis = 1)

print(X_test.shape)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')


(1139639, 17)
Cantidad de datos de train: 1865549
Cantidad de datos de validación: 466388


In [6]:
# Define the XGBoost classifier
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',  # You can also use 'logloss' for binary classification
                            use_label_encoder=False)
clf_xgb.fit(X_train, Y_train, verbose = True, eval_set = [(X_val, Y_val)])

[0]	validation_0-auc:0.79858
[1]	validation_0-auc:0.80458


/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:16:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[2]	validation_0-auc:0.80632
[3]	validation_0-auc:0.80853
[4]	validation_0-auc:0.80872
[5]	validation_0-auc:0.81044
[6]	validation_0-auc:0.81411
[7]	validation_0-auc:0.81510
[8]	validation_0-auc:0.81398
[9]	validation_0-auc:0.82069
[10]	validation_0-auc:0.82538
[11]	validation_0-auc:0.82665
[12]	validation_0-auc:0.82800
[13]	validation_0-auc:0.82888
[14]	validation_0-auc:0.83002
[15]	validation_0-auc:0.83026
[16]	validation_0-auc:0.83093
[17]	validation_0-auc:0.83119
[18]	validation_0-auc:0.83132
[19]	validation_0-auc:0.83131
[20]	validation_0-auc:0.83212
[21]	validation_0-auc:0.83216
[22]	validation_0-auc:0.83222
[23]	validation_0-auc:0.83293
[24]	validation_0-auc:0.83284
[25]	validation_0-auc:0.83290
[26]	validation_0-auc:0.83288
[27]	validation_0-auc:0.83288
[28]	validation_0-auc:0.83295
[29]	validation_0-auc:0.83361
[30]	validation_0-auc:0.83374
[31]	validation_0-auc:0.83394
[32]	validation_0-auc:0.83433
[33]	validation_0-auc:0.83427
[34]	validation_0-auc:0.83460
[35]	validation_0-

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [57]:
X_test['id'] = range(1, len(X_test) + 1)
y_pred_test = clf_xgb.predict_proba(X_test.drop(columns=["id"]))[:, clf_xgb.classes_ == 1].squeeze()
submission_df = pd.DataFrame({"id": X_test["id"], "Label": y_pred_test})
submission_df["id"] = submission_df["id"].astype(int)
submission_df.to_csv("XGBOOTS.csv", sep=",", index=False)